# 實驗：評估壓成3k以下的事實摘要和情節摘要。

In [1]:
import json
import time
import requests
import openai
import os
from dotenv import load_dotenv
from rouge_chinese import Rouge

load_dotenv(".env")
openai.api_key = os.getenv('openai_api_key')
url = os.getenv('breeze_url')
rouge = Rouge()

In [2]:
def get_breeze(input_content):
    system = 'You are a helpful AI assistant built by MediaTek Research. The user you are helping speaks Traditional Chinese and comes from Taiwan.'

    payload = json.dumps({
        "inputs": f"<s> {system} [INST] {input_content} [/INST]",
        "parameters": {
            "do_sample": True,
            "temperature": 0.01,
            "top_p": 0.95,
            'repetition_penalty': 1, # 不要一直生重複的，大約設1~1.2之間輸出是剛好的
            'max_new_tokens':1000, # 要設一下，不然會講到一半就停掉
        }
        })
    headers = {
    'Content-Type': 'application/json',
    'accept': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, data=payload)
        j_result = json.loads(response.text)
        if "generated_text" in j_result.keys():
            return j_result['generated_text']
        else:
            return j_result
    except Exception as e:
        print(e)

In [3]:
def generate_scenario(judgement):
    # 有範例的情節摘要
    example = '以下是一個情節摘要的範例，輸入是判決書內容，輸出是情節摘要。範例如下：輸入:孫桂蘭與孟奇於民國83年7月10日結婚,孫桂蘭係孟奇之子女即孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉之繼母。孟奇於96年1月22日死亡,孫桂蘭明知孟奇死亡後,其名下財產均為遺產,於遺產分割前屬孫桂蘭、孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉全體繼承人公同共有,未經全體繼承人同意,任一繼承人不得擅自處分,竟意圖為自己不法所有,基於詐欺取財及行使偽造私文書之犯意,於附表各編號所示之時間,至附表各編號所示金融機構,冒用孟奇之名義,在空白取款憑證填寫如附表各編號所示之日期、帳號及金額,並於「取款印鑑」欄,使用孟奇之印章盜蓋印文1枚,表示係孟奇欲自附表各編號所示金融機構帳戶內提領附表各編號所示提款金額,而偽造孟奇名義之取款憑證私文書後,再連同上開帳戶存摺持之向不知情之附表各編號所示金融機構承辦人員辦理取款而行使之,致該金融機構承辦人員陷於錯誤,誤認孫桂蘭係經孟奇本人授權而為上開手續,如數交付如附表各編號所示金額,均足以生損害於孟慶明、孟慶虹、孟慶經、孟慶緯(已歿)、孟蓉之權益及日盛國際商業銀行股份有限公司《下稱日盛商業銀行》、合作金庫商業銀行股份有限公司竹塹分行《下稱合作金庫商業銀行竹塹分行》、中華郵政股份有限公司清華大學郵局《下稱清華大學郵局》、永豐商業銀行股份有限公司《下稱永豐商業銀行》等金融機構對於孟奇帳戶資料、存款管理之正確性。 輸出:在這個案件中,涉及到一段婚姻關係,並有繼承人之間的財產分割。一名已故的個人的遺產成為了糾紛的焦點。在其去世後,其配偶及子女成為了繼承人。然而,配偶明知其去世,卻企圖以詐欺和偽造私文書的方式來獲取他們的共同財產。她冒用已故人的名義,填寫了取款憑證並偽造了他的印章,使金融機構誤以為是合法的取款要求。最終,金融機構執行了這些虛偽的指示,導致繼承人之間的權益受損,也對涉及的金融機構造成了損害。这些行為涉及偽造私文書、詐欺取財,並對繼承人和金融機構產生了不良影響。請學習以上範例，將以下判決書內容做情節摘要，'
    scenario_with_example_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。{example}判決書內容：輸入：{judgement}。輸出：'
    
    start = time.time()
    example_scenario = get_breeze(scenario_with_example_prompt)
    end = time.time()
    print(f'Generate Scenario W Example Time: {end - start:.5f}')
    print(f'Scenario w example Length: {len(example_scenario)}')
    example_scenario = example_scenario.replace('\n', '').replace('\t', '')

    # 沒範例的情節摘要
    scenario_without_example_prompt = f'我會給你一篇法律判決書內容，幫我做情節摘要，可能包含事實情節描述、判決結果及理由。判決書內容如下：{judgement}。'
    
    start = time.time()
    scenario = get_breeze(scenario_without_example_prompt)
    end = time.time()
    print(f'Generate Scenario W/O Example Time: {end - start:.5f}')
    print(f'Scenario w/o example Length: {len(scenario)}')
    scenario = scenario.replace('\n', '').replace('\t', '')

    return example_scenario, scenario

In [4]:
def split_passage(judgement):
    summaries = []

    judgement_len = len(judgement)
    window_size = 3000
    overlap_size = 100
    start = 0
    end = min(window_size, judgement_len)

    while start < judgement_len:
        passage = judgement[start:end]
        fact_summary_prompt = f'我會給你一篇法律判決書內容，幫我做事實摘要，可能包含事實情節描述。判決書內容如下：{passage}。'
        
        start = time.time()
        summary = get_breeze(fact_summary_prompt)
        end = time.time()
        print(f'Split Summary Time: {end - start:.5f}')

        summary = summary.replace('\n', '').replace('\t', '')
        summaries.append(summary)
        
        start += window_size - overlap_size
        end = min(start + window_size, judgement_len)

    summary_str = ''.join(summaries)

    return summary_str

In [5]:
def process_judgement(judgement):
    print(f'Origin Length: {len(judgement)}')
    if len(judgement) > 3000:
        print(f'Judgement processing...')
        fact_summary = split_passage(judgement)
        print(f'First summary length: {len(fact_summary)}')
        print(f'First summary complete!')
        print(f'Scenario generating...')
        scenario_with_example, scenario_without_example = generate_scenario(fact_summary)
        print(f'Scenario complete!')
        print(f'='*100)
        return {
            "judgement": judgement,
            "fact_summary": fact_summary,
            "with_example": scenario_with_example,
            "without_example": scenario_without_example
        }
    else:
        print(f'Scenario generating...')
        scenario_with_example, scenario_without_example = generate_scenario(judgement)
        print(f'Scenario complete!')
        print(f'='*100)
        return {
            "judgement": judgement,
            "fact_summary": "None",
            "with_example": scenario_with_example,
            "without_example": scenario_without_example
        }

In [6]:
def get_gpt(query, max_test_time = 3):
    system_prompt = "You are ChatGPT, a large language model trained by OpenAI."
    user_prompt = f"我會給你三個範例 Example 1, Example 2 and Example 3 (輸入:Text,輸出:Context Summary) Example1 : Text輸入:孫桂蘭與孟奇於民國83年7月10日結婚,孫桂蘭係孟奇之子女即孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉之繼母。孟奇於96年1月22日死亡,孫桂蘭明知孟奇死亡後,其名下財產均為遺產,於遺產分割前屬孫桂蘭、孟慶明、孟慶虹、孟慶經、孟慶緯、孟蓉全體繼承人公同共有,未經全體繼承人同意,任一繼承人不得擅自處分,竟意圖為自己不法所有,基於詐欺取財及行使偽造私文書之犯意,於附表各編號所示之時間,至附表各編號所示金融機構,冒用孟奇之名義,在空白取款憑證填寫如附表各編號所示之日期、帳號及金額,並於「取款印鑑」欄,使用孟奇之印章盜蓋印文1枚,表示係孟奇欲自附表各編號所示金融機構帳戶內提領附表各編號所示提款金額,而偽造孟奇名義之取款憑證私文書後,再連同上開帳戶存摺持之向不知情之附表各編號所示金融機構承辦人員辦理取款而行使之,致該金融機構承辦人員陷於錯誤,誤認孫桂蘭係經孟奇本人授權而為上開手續,如數交付如附表各編號所示金額,均足以生損害於孟慶明、孟慶虹、孟慶經、孟慶緯(已歿)、孟蓉之權益及日盛國際商業銀行股份有限公司《下稱日盛商業銀行》、合作金庫商業銀行股份有限公司竹塹分行《下稱合作金庫商業銀行竹塹分行》、中華郵政股份有限公司清華大學郵局《下稱清華大學郵局》、永豐商業銀行股份有限公司《下稱永豐商業銀行》等金融機構對於孟奇帳戶資料、存款管理之正確性。 Context Summary輸出:在這個案件中,涉及到一段婚姻關係,並有繼承人之間的財產分割。一名已故的個人的遺產成為了糾紛的焦點。在其去世後,其配偶及子女成為了繼承人。然而,配偶明知其去世,卻企圖以詐欺和偽造私文書的方式來獲取他們的共同財產。她冒用已故人的名義,填寫了取款憑證並偽造了他的印章,使金融機構誤以為是合法的取款要求。最終,金融機構執行了這些虛偽的指示,導致繼承人之間的權益受損,也對涉及的金融機構造成了損害。这些行為涉及偽造私文書、詐欺取財,並對繼承人和金融機構產生了不良影響。 Example 2 : Text輸入:丁○○○為陳全清之妻,己○○為陳全清之弟即甲○○之兒媳,乙○○、陳全清、甲○○、廖陳專、丙○○、陳素美、陳月娥(下稱乙○○等7人)均為被繼承人陳秋火之子女,緣陳秋火於民國107年1月27日死亡,乙○○等7人均為陳秋火之法定繼承人。丁○○○、己○○明知陳秋火業已死亡,無法再以其名義為任何法律行為,且陳秋火所遺留之存款由全體繼承人繼承,應以全體繼承人名義具領款項,或全體繼承人同意委任代理人,依據繼承之程序,始得提領款項,丁○○○、己○○竟未得乙○○之同意,共同基於行使偽造私文書之犯意聯絡,將其等保管之陳秋火之存摺、印章取出,於107年1月29日9時55分許,共同前往位於苗栗縣○○鎮○○路0號之玉山商業銀行股份有限公司(下稱玉山銀行)後龍分行,冒用陳秋火之名義,申辦:將陳秋火之玉山銀行後龍分行定期存款(帳號依序分別為000000000號、000000000號、000000000號,金額均為新臺幣(下同)10萬元,共計30萬元)解約,存入陳秋火之同銀行帳號為0000000000000號帳戶(下稱甲帳戶)後,甲帳戶餘額為102萬2325元,再自甲帳戶提領現金25萬元,並將餘額77萬2325元匯至不知情之陳永聰(為甲○○之子、己○○之夫,另經臺灣苗栗地方檢察署檢察官為不起訴處分)之同銀行帳號0000000000000號帳戶(下稱乙帳戶)等交易,復持上開存摺、印章交由不知情之玉山銀行後龍分行承辦人員盜蓋於內部傳票、取款憑條上(偽造之文件名稱、交易摘要及金額、欄位、盜蓋印章之印文及數量均詳如附表所載),以利用該承辦人員偽造屬私文書之上開內部傳票、取款憑條後,據以交由該承辦人員辦理而行使之,致使承辦之玉山銀行人員誤認陳秋火尚在世並授權,而將上開陳秋火之定期存款解約及存入甲帳戶內之存款102萬2325萬元憑摺支取,除其中25萬元提領現金外,餘77萬2325元匯入乙帳戶,足以生損害於陳秋火之繼承人及玉山銀行後龍分行對於存款帳戶管理之正確性。Context Summary輸出:這宗案件涉及家族關係與遺產問題。已故個人的遺產引發了爭議。他的配偶與子女是合法繼承人,但配偶卻企圖透過詐欺手段來侵占共同財產。她冒用已故人的身份,偽造文件並盗用印章,讓金融機構誤以為是合法的交易。最終,金融機構執行了這些虛偽指示,導致繼承人的權益受損,也給金融機構造成損失。這些行為牽涉到偽造文件和詐騙,對繼承人和金融機構都帶來不良影響。Example 3: Text輸入:吳文錦係吳清海(民國108年2月2日死亡)之長子,吳芸安則係吳清海次子吳文樑(於103年3月22日死亡)之女,為吳文樑之代位繼承人,吳淑妮、吳金桂、吳淑惠、吳月嬌亦為吳清海之子女,惟吳文錦於吳清海過世後,為辦理吳清海之後事相關事宜時,明知不得再以吳清海之名義提領存款,須經全體繼承人同意或授權,始得提領存款,竟未經全體繼承人之同意或授權,基於行使偽造私文書之犯意,接續於108年2月11日、108年2月20日,持其所保管吳清海之雲林縣○○鄉○○0○號:000-0000000000000號,下稱四湖鄉農會帳戶)之存摺、印鑑,前往雲林縣○○鄉○○村○○○路00號之雲林縣四湖鄉農會,冒用吳清海之名義,在該農會之取款憑條存戶簽章欄上,填寫領取新臺幣(下同)337,600元、7,290元,並偽簽「吳清海」之署押1枚,盜蓋「吳清海」印文2枚(詳如附表所示),而偽造吳清海同意提領337,600元、7,290元之私文書,並持該偽造之取款憑條交予上開農會之不知情承辦人員而行使之,使吳文錦自前開帳戶提領共344,890元,足生損害於吳清海全體繼承人及四湖鄉農會對客戶存款管理之正確性。Context Summary輸出:這宗案件涉及家族關係和遺產問題。已故個人的子女是合法繼承人,但其中一人明知不得未經全體繼承人同意或授權就提領存款。他使用偽造文件,包括存摺和印鑑,前往當地農會冒充已故父親的身份,提領款項,造成損害。这涉及偽造文件和可能的詐騙行為,對繼承人和農會的存款管理產生不良影響。 請你模仿Example 1 , Example 2, Example 3,來生成以下一個輸入(Text1)之context摘要,禁止出現人名。Text1 輸入:{query} Text1輸出:(請用繁體中文輸出)"
    
    for i in range(max_test_time):
        try:
            response = openai.chat.completions.create(
                model="gpt-4-turbo-preview",
                messages=[{"role": "system", "content": system_prompt},
                          {"role": "user", "content": user_prompt}],
                max_tokens=4096,
                temperature=1,
            )
            response_text = response.choices[0].message.content
            break
        except Exception as e:
            print(f"Error {i+1}: {str(e)}")
            response_text = "ERROR"
    return response_text

## 實驗一：看壓縮效果
* GPT摘要：origin fact -> (GPT) -> scenario
* Breeze摘要：origin fact -> (Breeze) -> scenario
* Breeze壓縮+摘要：origin fact -> (Breeze) -> summary fact -> (Breeze) -> scenario

In [ ]:
with open('dataset/train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

### (答)GPT摘要：origin fact -> (GPT) -> scenario

In [ ]:
processed_data = []

for data in train_data[:10]:
    judgement = data['fact']
    print(f'Scenario generating...')
    summary = get_gpt(judgement)
    print(f'Summayr: {summary}')
    print(f'Scenario complete!')
    print(f'='*100)
    
    temp = {
        "judgement": judgement,
        "summary": summary
    }

    processed_data.append(temp)

with open("experiment1/gpt_scenario.json", "w", encoding="utf-8") as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

### Breeze摘要：origin fact -> (Breeze) -> scenario

In [9]:
processed_data = []

for data in train_data[:10]:
    judgement = data['fact']
    print(f'Scenario generating...')
    scenario_with_example, scenario_without_example = generate_scenario(judgement)
    scenario_with_example = scenario_with_example.replace('\n', '').replace('\t', '')
    scenario_without_example = scenario_without_example.replace('\n', '').replace('\t', '')
    print(f'Scenario complete!')
    print(f'='*100)
    
    
    temp = {
        "judgement": judgement,
        "scenario_with_example": scenario_with_example,
        "scenario_without_example": scenario_without_example
        }
    
    processed_data.append(temp)

with open("experiment1/breeze_scenario.json", "w", encoding="utf-8") as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

Scenario generating...
Generate Scenario W Example Time: 52.93324
Scenario w example Length: 1358
Generate Scenario W/O Example Time: 22.91378
Scenario w/o example Length: 602
Scenario complete!
Scenario generating...
Generate Scenario W Example Time: 7.34081
Scenario w example Length: 222
Generate Scenario W/O Example Time: 16.76494
Scenario w/o example Length: 545
Scenario complete!
Scenario generating...
Generate Scenario W Example Time: 5.86711
Scenario w example Length: 187
Generate Scenario W/O Example Time: 9.43987
Scenario w/o example Length: 371
Scenario complete!
Scenario generating...
Generate Scenario W Example Time: 64.30944
Scenario w example Length: 1318
Generate Scenario W/O Example Time: 61.65051
Scenario w/o example Length: 1418
Scenario complete!
Scenario generating...
Generate Scenario W Example Time: 61.63407
Scenario w example Length: 1248
Generate Scenario W/O Example Time: 59.32629
Scenario w/o example Length: 1194
Scenario complete!
Scenario generating...
Gener

### Breeze壓縮+摘要：origin fact -> (Breeze) -> summary fact -> (Breeze) -> scenario

In [13]:
processed_data = []

for data in train_data[:10]:
    judgement = data['fact']
    processed_data.append(process_judgement(judgement))

with open("experiment1/breeze_summary_scenario.json", "w", encoding="utf-8") as j:
    json.dump(processed_data, j, ensure_ascii=False, indent=4)

Origin Length: 6546
Judgement processing...
Split Summary Time: 12.74788
First summary length: 348
First summary complete!
Scenario generating...
Generate Scenario W Example Time: 6.94026
Scenario w example Length: 203
Generate Scenario W/O Example Time: 9.73048
Scenario w/o example Length: 285
Scenario complete!
Origin Length: 3385
Judgement processing...
Split Summary Time: 23.60200
First summary length: 602
First summary complete!
Scenario generating...
Generate Scenario W Example Time: 16.06940
Scenario w example Length: 512
Generate Scenario W/O Example Time: 20.54658
Scenario w/o example Length: 674
Scenario complete!
Origin Length: 1331
Scenario generating...
Generate Scenario W Example Time: 7.09446
Scenario w example Length: 187
Generate Scenario W/O Example Time: 12.14091
Scenario w/o example Length: 373
Scenario w example Length: 187
Scenario w/o example Length: 367
Scenario complete!
Origin Length: 10172
Judgement processing...
Split Summary Time: 25.14320
First summary len

KeyboardInterrupt: 

### 合併資料

In [12]:
with open('experiment1/gpt_scenario.json', 'r', encoding='utf-8') as f:
    gpt_scenario = json.load(f)

with open('experiment1/breeze_scenario.json', 'r', encoding='utf-8') as f:
    breeze_scenario = json.load(f)

with open('experiment1/breeze_summary_scenario.json', 'r', encoding='utf-8') as f:
    breeze_summary_scenario = json.load(f)

In [14]:
print(len(gpt_scenario))
col = list(gpt_scenario[0].keys())
print(col)
gpt_scenario[0]

10
['judgement', 'summary']


{'judgement': '一、陳泰龍明知其與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM，陳玉婉涉嫌本案偽造文書部分，另經臺灣臺南地方法院檢察署檢察官通緝中）間並無結婚之真意，然因詹巧薇、張晉源等人欲找臺灣男子前往泰國與泰國女子辦理假結婚，林清水負責介紹臺灣男子予詹巧薇、張晉源等人以賺取新臺幣（下同）1萬元之仲介費，適陳泰龍依林清水於報紙上刊登之徵人廣告應徵時，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬，迨陳泰龍應允同意後，林清水將其轉介紹予詹巧薇、張晉源等人，陳泰龍遂與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，先由陳泰龍依指示於民國93年4月3日前往泰國，並於同年月7日與陳玉婉在泰國曼谷市法拉坎農區註冊辦事處登記結婚，取得上開註冊辦事處核發之結婚登記書、結婚證書等文件後，陳泰龍即行返臺，並於同年5月19日持經我國駐泰國臺北經濟文化辦事處認證之上開結婚登記書、結婚證書等資料，至臺南縣永康市（改制前稱謂，以下沿用舊制稱謂）戶政事務所填寫結婚登記申請書，申請辦理其與陳玉婉之結婚登記，因此使上開戶政事務所僅有形式審查權之承辦公務員陷於錯誤，於形式上審核後，將陳泰龍與陳玉婉於93年4月7日結婚之不實事項，登載於其職務上所掌屬公文書之戶籍登記資料內，並據以核發戶籍謄本，而足以生損害於戶政機關對於戶政管理及戶籍登記資料維護之正確性。迨陳玉婉來臺後，為取得居留身分，再由陳泰龍、陳玉婉承前概括之犯意聯絡，於同年6月14日及94年5月5日，均持前開登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期而連續行使之（業經檢察官當庭補充），經承辦人員實質審查結果，均未發覺陳泰龍與陳玉婉假結婚之實情，而發給陳玉婉居留證或核准延期，各足以生損害於主管機關對外僑居留管理之正確性。二、案經臺南市警察局（現改制為臺南市政府警察局，以下沿用舊制稱謂）第一分局報告臺灣臺南地方法院檢察署檢察官偵查起訴。理由甲、程序方面：一、本件以下所引用具傳聞證據性質之供述證據，因本件被告陳泰龍、檢察官等人於本院準備程序進行時，均未爭執前開陳述之證據能力，且迄至本案言詞辯論終結前亦均未聲明異議，應視為已同意援引作為證據，又

In [15]:
print(len(breeze_scenario))
col = list(breeze_scenario[0].keys())
print(col)
breeze_scenario[0]

10
['judgement', 'scenario_with_example', 'scenario_without_example']


{'judgement': '一、陳泰龍明知其與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM，陳玉婉涉嫌本案偽造文書部分，另經臺灣臺南地方法院檢察署檢察官通緝中）間並無結婚之真意，然因詹巧薇、張晉源等人欲找臺灣男子前往泰國與泰國女子辦理假結婚，林清水負責介紹臺灣男子予詹巧薇、張晉源等人以賺取新臺幣（下同）1萬元之仲介費，適陳泰龍依林清水於報紙上刊登之徵人廣告應徵時，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬，迨陳泰龍應允同意後，林清水將其轉介紹予詹巧薇、張晉源等人，陳泰龍遂與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，先由陳泰龍依指示於民國93年4月3日前往泰國，並於同年月7日與陳玉婉在泰國曼谷市法拉坎農區註冊辦事處登記結婚，取得上開註冊辦事處核發之結婚登記書、結婚證書等文件後，陳泰龍即行返臺，並於同年5月19日持經我國駐泰國臺北經濟文化辦事處認證之上開結婚登記書、結婚證書等資料，至臺南縣永康市（改制前稱謂，以下沿用舊制稱謂）戶政事務所填寫結婚登記申請書，申請辦理其與陳玉婉之結婚登記，因此使上開戶政事務所僅有形式審查權之承辦公務員陷於錯誤，於形式上審核後，將陳泰龍與陳玉婉於93年4月7日結婚之不實事項，登載於其職務上所掌屬公文書之戶籍登記資料內，並據以核發戶籍謄本，而足以生損害於戶政機關對於戶政管理及戶籍登記資料維護之正確性。迨陳玉婉來臺後，為取得居留身分，再由陳泰龍、陳玉婉承前概括之犯意聯絡，於同年6月14日及94年5月5日，均持前開登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期而連續行使之（業經檢察官當庭補充），經承辦人員實質審查結果，均未發覺陳泰龍與陳玉婉假結婚之實情，而發給陳玉婉居留證或核准延期，各足以生損害於主管機關對外僑居留管理之正確性。二、案經臺南市警察局（現改制為臺南市政府警察局，以下沿用舊制稱謂）第一分局報告臺灣臺南地方法院檢察署檢察官偵查起訴。理由甲、程序方面：一、本件以下所引用具傳聞證據性質之供述證據，因本件被告陳泰龍、檢察官等人於本院準備程序進行時，均未爭執前開陳述之證據能力，且迄至本案言詞辯論終結前亦均未聲明異議，應視為已同意援引作為證據，又

In [16]:
print(len(breeze_summary_scenario))
col = list(breeze_summary_scenario[0].keys())
print(col)
breeze_summary_scenario[0]

50
['judgement', 'fact_summary', 'with_example', 'without_example']


{'judgement': '一、陳泰龍明知其與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM，陳玉婉涉嫌本案偽造文書部分，另經臺灣臺南地方法院檢察署檢察官通緝中）間並無結婚之真意，然因詹巧薇、張晉源等人欲找臺灣男子前往泰國與泰國女子辦理假結婚，林清水負責介紹臺灣男子予詹巧薇、張晉源等人以賺取新臺幣（下同）1萬元之仲介費，適陳泰龍依林清水於報紙上刊登之徵人廣告應徵時，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬，迨陳泰龍應允同意後，林清水將其轉介紹予詹巧薇、張晉源等人，陳泰龍遂與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，先由陳泰龍依指示於民國93年4月3日前往泰國，並於同年月7日與陳玉婉在泰國曼谷市法拉坎農區註冊辦事處登記結婚，取得上開註冊辦事處核發之結婚登記書、結婚證書等文件後，陳泰龍即行返臺，並於同年5月19日持經我國駐泰國臺北經濟文化辦事處認證之上開結婚登記書、結婚證書等資料，至臺南縣永康市（改制前稱謂，以下沿用舊制稱謂）戶政事務所填寫結婚登記申請書，申請辦理其與陳玉婉之結婚登記，因此使上開戶政事務所僅有形式審查權之承辦公務員陷於錯誤，於形式上審核後，將陳泰龍與陳玉婉於93年4月7日結婚之不實事項，登載於其職務上所掌屬公文書之戶籍登記資料內，並據以核發戶籍謄本，而足以生損害於戶政機關對於戶政管理及戶籍登記資料維護之正確性。迨陳玉婉來臺後，為取得居留身分，再由陳泰龍、陳玉婉承前概括之犯意聯絡，於同年6月14日及94年5月5日，均持前開登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期而連續行使之（業經檢察官當庭補充），經承辦人員實質審查結果，均未發覺陳泰龍與陳玉婉假結婚之實情，而發給陳玉婉居留證或核准延期，各足以生損害於主管機關對外僑居留管理之正確性。二、案經臺南市警察局（現改制為臺南市政府警察局，以下沿用舊制稱謂）第一分局報告臺灣臺南地方法院檢察署檢察官偵查起訴。理由甲、程序方面：一、本件以下所引用具傳聞證據性質之供述證據，因本件被告陳泰龍、檢察官等人於本院準備程序進行時，均未爭執前開陳述之證據能力，且迄至本案言詞辯論終結前亦均未聲明異議，應視為已同意援引作為證據，又

In [11]:
data = []

for idx in range(len(gpt_scenario)):
    judgement = gpt_scenario[idx]["judgement"]
    length = len(judgement)
    gpt = gpt_scenario[idx]["summary"]
    breeze1 = breeze_scenario[idx]["scenario_with_example"]
    breeze2 = breeze_summary_scenario[idx]["with_example"]

    data.append({
        "judgement": judgement,
        "length": length,
        "gpt": gpt,
        "breeze1": breeze1,
        "breeze2": breeze2
    })

with open("experiment1/scenario_merge.json", "w", encoding="utf-8") as j:
    json.dump(data, j, ensure_ascii=False, indent=4)

### 評估摘要結果
* (答)GPT摘要：origin fact -> (GPT) -> scenario
* Breeze摘要：origin fact -> (Breeze) -> scenario
* Breeze壓縮+摘要：origin fact -> (Breeze) -> summary fact -> (Breeze) -> scenario

In [7]:
with open('experiment1/scenario_merge.json', 'r', encoding='utf-8') as f:
    merge_data = json.load(f)

merge_data[0]

{'judgement': '一、陳泰龍明知其與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM，陳玉婉涉嫌本案偽造文書部分，另經臺灣臺南地方法院檢察署檢察官通緝中）間並無結婚之真意，然因詹巧薇、張晉源等人欲找臺灣男子前往泰國與泰國女子辦理假結婚，林清水負責介紹臺灣男子予詹巧薇、張晉源等人以賺取新臺幣（下同）1萬元之仲介費，適陳泰龍依林清水於報紙上刊登之徵人廣告應徵時，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬，迨陳泰龍應允同意後，林清水將其轉介紹予詹巧薇、張晉源等人，陳泰龍遂與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，先由陳泰龍依指示於民國93年4月3日前往泰國，並於同年月7日與陳玉婉在泰國曼谷市法拉坎農區註冊辦事處登記結婚，取得上開註冊辦事處核發之結婚登記書、結婚證書等文件後，陳泰龍即行返臺，並於同年5月19日持經我國駐泰國臺北經濟文化辦事處認證之上開結婚登記書、結婚證書等資料，至臺南縣永康市（改制前稱謂，以下沿用舊制稱謂）戶政事務所填寫結婚登記申請書，申請辦理其與陳玉婉之結婚登記，因此使上開戶政事務所僅有形式審查權之承辦公務員陷於錯誤，於形式上審核後，將陳泰龍與陳玉婉於93年4月7日結婚之不實事項，登載於其職務上所掌屬公文書之戶籍登記資料內，並據以核發戶籍謄本，而足以生損害於戶政機關對於戶政管理及戶籍登記資料維護之正確性。迨陳玉婉來臺後，為取得居留身分，再由陳泰龍、陳玉婉承前概括之犯意聯絡，於同年6月14日及94年5月5日，均持前開登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期而連續行使之（業經檢察官當庭補充），經承辦人員實質審查結果，均未發覺陳泰龍與陳玉婉假結婚之實情，而發給陳玉婉居留證或核准延期，各足以生損害於主管機關對外僑居留管理之正確性。二、案經臺南市警察局（現改制為臺南市政府警察局，以下沿用舊制稱謂）第一分局報告臺灣臺南地方法院檢察署檢察官偵查起訴。理由甲、程序方面：一、本件以下所引用具傳聞證據性質之供述證據，因本件被告陳泰龍、檢察官等人於本院準備程序進行時，均未爭執前開陳述之證據能力，且迄至本案言詞辯論終結前亦均未聲明異議，應視為已同意援引作為證據，又

In [8]:
# GPT vs Breeze1
scores_list_1 = []

for data in merge_data:
    answer = data['gpt']
    ref1 = data['breeze1']

    answer = ' '.join(answer)
    ref1 = ' '.join(ref1)
    # print(f'answer: {answer}\nref: {ref1}')

    scores = rouge.get_scores(answer, ref1)
    rouge_1_r = round(scores[0]['rouge-1']['r'], 3)
    rouge_2_r = round(scores[0]['rouge-2']['r'], 3)
    rouge_l_r = round(scores[0]['rouge-l']['r'], 3)

    rouge_1_p = round(scores[0]['rouge-1']['p'], 3)
    rouge_2_p = round(scores[0]['rouge-2']['p'], 3)
    rouge_l_p = round(scores[0]['rouge-l']['p'], 3)

    scores_list_1.append((rouge_1_r, rouge_2_r, rouge_l_r, rouge_1_p, rouge_2_p, rouge_l_p))

In [9]:
# GPT vs Breeze2
scores_list_2 = []

for data in merge_data:
    answer = data['gpt']
    ref2 = data['breeze2']

    answer = ' '.join(answer)
    ref2 = ' '.join(ref2)
    # print(f'answer: {answer}\nref: {ref2}')

    scores = rouge.get_scores(answer, ref2)
    rouge_1_r = round(scores[0]['rouge-1']['r'], 3)
    rouge_2_r = round(scores[0]['rouge-2']['r'], 3)
    rouge_l_r = round(scores[0]['rouge-l']['r'], 3)

    rouge_1_p = round(scores[0]['rouge-1']['p'], 3)
    rouge_2_p = round(scores[0]['rouge-2']['p'], 3)
    rouge_l_p = round(scores[0]['rouge-l']['p'], 3)

    scores_list_2.append((rouge_1_r, rouge_2_r, rouge_l_r, rouge_1_p, rouge_2_p, rouge_l_p))

In [10]:
len_list = []
for data in merge_data:
    len_list.append(data['length'])

In [11]:
print(f'{"="*46} GPT vs Breeze 1 {"="*46}')
print(f'{"="*30} Recall {"="*40} Precision {"="*20}')
print(f'idx\tlength\t\tR-1\t\tR-2\t\tR-L\t|\tR-1\t\tR-2\t\tR-L')

for idx, scores in enumerate(scores_list_1):
    rouge_1_r, rouge_2_r, rouge_l_r, rouge_1_p, rouge_2_p, rouge_l_p = scores
    print(f'{idx}\t{len_list[idx]}\t\t{rouge_1_r}\t\t{rouge_2_r}\t\t{rouge_l_r}\t|\t{rouge_1_p}\t\t{rouge_2_p}\t\t{rouge_l_p}')

print(f'{"="*46} GPT vs Breeze 2 {"="*46}')
print(f'{"="*30} Recall {"="*40} Precision {"="*20}')
print(f'idx\tlength\t\tR-1\t\tR-2\t\tR-L\t|\tR-1\t\tR-2\t\tR-L')

for idx, scores in enumerate(scores_list_2):
    rouge_1_r, rouge_2_r, rouge_l_r, rouge_1_p, rouge_2_p, rouge_l_p = scores
    print(f'{idx}\t{len_list[idx]}\t\t{rouge_1_r}\t\t{rouge_2_r}\t\t{rouge_l_r}\t|\t{rouge_1_p}\t\t{rouge_2_p}\t\t{rouge_l_p}')

============================================== GPT vs Breeze 1 ==============================================
============================== Recall ======================================== Precision ====================
idx	length		R-1		R-2		R-L	|	R-1		R-2		R-L
0	6546		0.461		0.132		0.045	|	0.379		0.109		0.226
1	3385		0.637		0.232		0.294	|	0.439		0.142		0.209
2	1331		0.495		0.226		0.296	|	0.413		0.179		0.232
3	10172		0.545		0.17		0.044	|	0.142		0.028		0.119
4	9266		0.365		0.098		0.047	|	0.2		0.05		0.251
5	13669		0.333		0.0		0.005	|	0.015		0.0		0.019
6	328		0.738		0.358		0.617	|	0.274		0.099		0.169
7	7083		0.175		0.0		0.009	|	0.059		0.0		0.041
8	8806		0.412		0.097		0.066	|	0.197		0.039		0.189
9	12726		0.214		0.042		0.014	|	0.023		0.005		0.059
============================================== GPT vs Breeze 2 ==============================================
============================== Recall ======================================== Precision ====================
idx	length		R-1		R-2		R-L	|	

### 合併資料

In [13]:
breeze_summary_scenario[0]['judgement']
breeze_summary_scenario[0]['fact_summary']

' 判決書摘要：\n被告陳泰龍明知與泰國籍女子陳玉婉（泰國名字為JARUWANWONGPRATOOM）間並無結婚之真意，仍與林清水、詹巧薇、張晉源等人共同基於行使使公務員登載不實公文書之概括犯意聯絡，辦理假結婚。被告依林清水於報紙上刊登之徵人廣告應徵，由林清水告知泰國女子欲以假結婚之方式申請來臺居留工作，其若擔任假結婚之人頭老公，除不用負擔前往泰國之食宿、機票費用外，尚可獲取5、6萬元之報酬。被告與林清水、詹巧薇、張晉源等人共同辦理假結婚，使公務員陷於錯誤，登載不實事項於戶籍登記資料內，並核發戶籍謄本。被告與陳玉婉辦理假結婚後，陳玉婉來臺後，被告與陳玉婉共同持登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之外僑居留證及申請居留證延期，使主管機關對外僑居留管理之正確性受損。 事實摘要：\n\n蔡明峰、林清水、詹巧薇和張晉源涉嫌擔任人頭老公與泰國女子假結婚。蔡明峰因本案被告與共犯陳玉婉辦理假結婚，涉嫌共同行使使公務員登載不實文書，經臺灣臺南地方法院檢察官認為與前開95年度簡上字第330號、99年度簡字第1313號論罪科刑部分有連續犯之裁判上一罪關係，故均以99年度偵字第17571、17771號、100年度偵字第4564號為不起訴處分確定。被告與共犯林清水並無仇怨，被告提供一部機車予共犯林清水使用。被告與共犯陳玉婉確係假結婚，被告的辯解為卸責之詞。被告適用行為時即修正前刑法之相關規定，對其較為有利，自應一體適用修正前刑法規定予以處斷。被告於93年5月19日、93年6月14日及94年5月5日先後多次行使使公務員登載不實文書之犯行，時間密接，所犯係構成要件相同之罪名，顯係基於概括犯意反覆為之，為連續犯，應依修正前刑法第56條規定論以一罪。 事實摘要：被告涉嫌於96年5月28日及96年9月3日與共犯陳玉婉共同持登載不實之戶籍謄本等資料，先後向臺南縣政府警察局外事課申請核發陳玉婉之延期居留等事宜。被告涉嫌共同行使使公務員登載不實文書犯行，均在95年7月1日刑法修正廢除第56條連續犯規定以後，未記載於起訴書犯罪事實內。由於此部分犯行在刑法修正後，未構成連續犯，故非本院得審判之範圍。'

In [27]:
test_summary = breeze_summary_scenario[:10]
len(test_summary)

10

In [31]:
processed_data = []

for idx, data in enumerate(test_summary):
    gpt = gpt_scenario[idx]["summary"]
    judgement = data['fact_summary']
    print(f'Scenario generating...')
    scenario = get_gpt(judgement)
    print(f'Scenario complete!')
    print(f'='*100)

    temp = {
        "judgement": judgement,
        "answer": gpt,
        "scenario": scenario
    }

    processed_data.append(temp)

with open("experiment1/summary_experiment.json", "w", encoding="utf-8") as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!
Scenario generating...
Scenario complete!


### 評估摘要好不好
* (答)GPT摘要：origin fact -> (GPT) -> scenario
* Breeze壓縮+摘要：origin fact -> (Breeze) -> summary fact -> (GPT) -> scenario

In [12]:
with open('experiment1/summary_experiment.json', 'r', encoding='utf-8') as f:
    summary_data = json.load(f)

In [16]:
# scenario(GPT) vs summary(breeze+GPT)
scores_list = []

for data in summary_data:
    answer = data['answer']
    ref1 = data['scenario']
    ref1 = ref1.replace('\n', '').replace('\t', '')

    answer = ' '.join(answer)
    ref1 = ' '.join(ref1)
    # print(f'answer: {answer}\nref: {ref1}')

    scores = rouge.get_scores(answer, ref1)
    rouge_1_r = round(scores[0]['rouge-1']['r'], 3)
    rouge_2_r = round(scores[0]['rouge-2']['r'], 3)
    rouge_l_r = round(scores[0]['rouge-l']['r'], 3)

    rouge_1_p = round(scores[0]['rouge-1']['p'], 3)
    rouge_2_p = round(scores[0]['rouge-2']['p'], 3)
    rouge_l_p = round(scores[0]['rouge-l']['p'], 3)

    scores_list.append((rouge_1_r, rouge_2_r, rouge_l_r, rouge_1_p, rouge_2_p, rouge_l_p))

In [17]:
print(f'{"="*46} GPT vs Breeze+GPT {"="*46}')
print(f'{"="*30} Recall {"="*40} Precision {"="*20}')
print(f'idx\tlength\t\tR-1\t\tR-2\t\tR-L\t|\tR-1\t\tR-2\t\tR-L')

for idx, scores in enumerate(scores_list):
    rouge_1_r, rouge_2_r, rouge_l_r, rouge_1_p, rouge_2_p, rouge_l_p = scores
    print(f'{idx}\t{len_list[idx]}\t\t{rouge_1_r}\t\t{rouge_2_r}\t\t{rouge_l_r}\t|\t{rouge_1_p}\t\t{rouge_2_p}\t\t{rouge_l_p}')

============================================== GPT vs Breeze+GPT ==============================================
============================== Recall ======================================== Precision ====================
idx	length		R-1		R-2		R-L	|	R-1		R-2		R-L
0	6546		0.553		0.228		0.299	|	0.557		0.222		0.292
1	3385		0.584		0.227		0.32	|	0.549		0.212		0.309
2	1331		0.294		0.063		0.184	|	0.119		0.021		0.059
3	10172		0.606		0.265		0.271	|	0.556		0.196		0.173
4	9266		0.481		0.229		0.298	|	0.548		0.233		0.303
5	13669		0.481		0.18		0.202	|	0.591		0.229		0.269
6	328		0.37		0.045		0.237	|	0.177		0.016		0.082
7	7083		0.471		0.161		0.216	|	0.542		0.184		0.257
8	8806		0.53		0.183		0.236	|	0.563		0.183		0.231
9	12726		0.517		0.207		0.194	|	0.702		0.291		0.286


## 實驗二：看摘要效果
* GPT摘要：origin fact -> (GPT) -> scenario
* Breeze壓縮+摘要：origin fact -> (Breeze) -> summary fact -> (Breeze) -> scenario
* Breeze壓縮+GPT摘要：origin fact -> (Breeze) -> summary fact -> (GPT) -> scenario